In [68]:
import numpy as np
import pandas as pd
from plotly import graph_objects as go
import torch
from torch import nn

In [69]:
# 准备数据

#正负样本数量
n_positive, n_negative = 2000, 2000
#生成正样本, 小圆环分布
r_p = 5.0 + torch.normal(0.0, 1.0, size=[n_positive, 1])
theta_p = 2 * np.pi * torch.rand([n_positive, 1])
Xp = torch.cat([r_p * torch.cos(theta_p), r_p * torch.sin(theta_p)], axis=1)
Yp = torch.ones_like(r_p)
#生成负样本, 大圆环分布
r_n = 8.0 + torch.normal(0.0, 1.0, size=[n_negative, 1])
theta_n = 2 * np.pi * torch.rand([n_negative, 1])
Xn = torch.cat([r_n * torch.cos(theta_n), r_n * torch.sin(theta_n)], axis=1)
Yn = torch.zeros_like(r_n)
#汇总样本
X = torch.cat([Xp, Xn], axis=0)
Y = torch.cat([Yp, Yn], axis=0)

In [70]:
# 预览
fig = go.Figure()
trace1 = go.Scatter(x=Xp[:, 0], y=Xp[:, 1], mode='markers', name='p')
trace2 = go.Scatter(x=Xn[:, 0], y=Xn[:, 1], mode='markers', name='n')
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=10,
        r=10,
        b=100,
        t=100,
        pad=3
    ),
)
fig.show()

## 低阶API

In [71]:
# 构建数据迭代器
def data_iter(features: torch.Tensor, labels, batch_size=8):
    num_exampls = len(features)
    idx = list(range(num_exampls))
    np.random.shuffle(idx)
    for i in range(0, num_exampls, batch_size):
        new_idx = torch.LongTensor(idx[i:min(i + batch_size, num_exampls)])
        yield features.index_select(0, new_idx), labels.index_select(0, new_idx)

(features, labels) = next(data_iter(X, Y))
print('features: ', features)
print('labels: ', labels)

features:  tensor([[ 0.0100,  6.9156],
        [ 5.3783, -1.6352],
        [-4.7164, -2.2320],
        [ 2.7553, -1.5370],
        [ 4.3743, -5.0449],
        [-7.6252, -3.2998],
        [ 6.8307, -6.3168],
        [-1.5446, -5.2486]])
labels:  tensor([[0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.]])


In [72]:
# 定义模型
class DNNModel(nn.Module):
    def __init__(self):
        super(DNNModel, self).__init__()
        self.w1 = nn.Parameter(torch.randn(2, 4))
        self.b1 = nn.Parameter(torch.zeros(1, 4))
        self.w2 = nn.Parameter(torch.randn(4, 8))
        self.b2 = nn.Parameter(torch.zeros(1, 8))
        self.w3 = nn.Parameter(torch.randn(8, 1))
        self.b3 = nn.Parameter(torch.zeros(1, 1))

    def forward(self, x):
        x = torch.relu(x@self.w1 + self.b1)
        x = torch.relu(x@self.w2 + self.b2)
        y = torch.sigmoid(x@self.w3 + self.b3)
        return y

    # 二元交叉熵
    def loss_func(self, y_pred, y_true):
        # 避免log(0) 错误
        eps = 1e-7
        y_pred = torch.clamp(y_pred, eps, 1.0-eps)
        bce = -y_true * torch.log(y_pred) - (1 - y_true) * torch.log(1 - y_pred)
        return torch.mean(bce)

    def metric_func(self, y_pred, y_true):
        y_pred = torch.where(y_pred > 0.5, torch.ones_like(y_pred, dtype=torch.float32),
                             torch.zeros_like(y_pred, dtype=torch.float32))
        acc = torch.mean(1 - torch.abs(y_true - y_pred))
        return acc


model = DNNModel()

In [73]:
# 测试结构
batch_size = 10
(features, labels) = next(data_iter(X, Y, batch_size))
predictions = model(features)

loss = model.loss_func(labels, predictions)
metric = model.metric_func(labels, predictions)
print("init loss:", loss.item())
print("init metric:", metric.item())

init loss: 9.562487602233887
init metric: 0.4001845717430115


In [74]:
def train_step(model, features, labels):
    # 正向
    predictions = model.forward(features)
    loss = model.loss_func(predictions, labels)
    metric = model.metric_func(predictions, labels)

    # 反向
    loss.backward()

    # 更新参数
    for param in model.parameters():
        param.data = (param.data - 0.01 * param.grad.data)

    # 梯度清零
    model.zero_grad()
    return loss.item(), metric.item()

# (features, labels) = next(data_iter(X, Y, batch_size))
# loss, metric = train_step(model, features, labels)
# print(loss, metric)


def train_model(model, epochs):
    for epoch in range(1, epochs+1):
        loss_list, metric_list = [], []
        for features, labels in data_iter(X, Y, batch_size):
            lossi, metrici = train_step(model, features, labels)
            loss_list.append(lossi)
            metric_list.append(metrici)
        loss = np.mean(loss_list)
        metric = np.mean(metric_list)

        if epoch % 100 == 0:
            print('epoch={} loss={} metric={} '.format(epoch, loss, metric))


train_model(model, epochs=500)

epoch=100 loss=0.18610517101595178 metric=0.9294999922811985 
epoch=200 loss=0.18402794773457573 metric=0.9252499924600124 
epoch=300 loss=0.1774154716054909 metric=0.9284999933838844 
epoch=400 loss=0.17851714089163578 metric=0.9292499920725823 
epoch=500 loss=0.17628757865400985 metric=0.928999992609024 


In [75]:
# 预测
# X_pred = torch.squeeze(model.forward(X))
Xp_pred = X[torch.squeeze(model.forward(X)>=0.5)]
Xn_pred = X[torch.squeeze(model.forward(X)<0.5)]
Xp_pred

tensor([[-2.1195,  4.4397],
        [ 3.2330, -5.0898],
        [-4.6811,  2.3082],
        ...,
        [-3.8134, -5.3091],
        [-0.3192, -6.3915],
        [-1.2381, -6.3085]])

In [76]:
# 预览
from plotly.subplots import make_subplots

fig = go.Figure()
trace_Xp = go.Scatter(x=Xp[:, 0], y=Xp[:, 1], mode='markers', name='p')
trace_Xn = go.Scatter(x=Xn[:, 0], y=Xn[:, 1], mode='markers', name='n')

trace_Xp_pred = go.Scatter(x=Xp_pred[:, 0], y=Xp_pred[:, 1], mode='markers', name='Xp_pre')
trace_Xn_pred = go.Scatter(x=Xn_pred[:, 0], y=Xn_pred[:, 1], mode='markers', name='Xn_pre')

# fig.add_trace(trace_Xp)
# fig.add_trace(trace_Xn)

fig = make_subplots(rows=1,  # 将画布分为两行
            cols=2,  # 将画布分为两列
            subplot_titles=['y_true','y_pred'],# 子图的标题
            x_title="x",
            y_title="y"
            )

fig.add_trace(trace_Xp,1,1)
fig.add_trace(trace_Xn,1,1)
fig.add_trace(trace_Xp_pred,1,2)
fig.add_trace(trace_Xn_pred,1,2)

fig.show()

## 中阶API

In [77]:
print(X.shape)
print(Y.shape)

torch.Size([4000, 2])
torch.Size([4000, 1])


In [94]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.nn import functional as F
ds = TensorDataset(X, Y)
dl = DataLoader(ds,batch_size=10,num_workers=2,shuffle=True)

In [109]:
class DNNModel(nn.Module):
    def __init__(self):
        super(DNNModel,self ).__init__()
        self.fc1 = nn.Linear(2,5)
        self.fc2 = nn.Linear(5,9)
        self.fc3 = nn.Linear(9,1)
    
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=nn.Sigmoid()(self.fc3(x))
        return x
    
    def loss_func(self, y_pred, y_true):
        return nn.BCELoss()(y_pred, y_true)

    def metric_func(self,y_pred,y_true):
        y_pred = torch.where(y_pred>0.5, torch.ones_like(y_pred,dtype=torch.float32), torch.zeros_like(y_pred,dtype=torch.float32))
        acc = torch.mean(1-torch.abs(y_true-y_pred))
        return acc
    
    @property
    def optimizer(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

model = DNNModel()

In [110]:
# 测试模型
(features, labels) = next(iter(dl))
predictions = model(features)

loss = model.loss_func(predictions, labels)
metric = model.metric_func(predictions, labels)

print('loss=',loss.item())
print('metric=',metric.item())

loss= 0.8797765970230103
metric= 0.20000000298023224


In [112]:
def train_step(model, features, labels):
    # 正向
    predictions = model(features)
    loss = model.loss_func(predictions, labels)
    metric = model.metric_func(predictions, labels)

    # 反向
    loss.backward()

    # 参数更新
    model.optimizer.step()
    model.optimizer.zero_grad()

    return loss.item(), metric.item()

features,labels = next(iter(dl))
train_step(model,features,labels)

(0.7293962240219116, 0.30000001192092896)

In [113]:
def train_model(model, epochs):
    for epoch in range(1, epochs+1):
        list_loss,list_metric=[],[]
        for features, labels in dl:
            lossi,metrici = train_step(model, features, labels)
            list_loss.append(lossi)
            list_metric.append(metrici)
        loss = np.mean(list_loss)
        metric = np.mean(list_metric)

        if epoch % 100 == 0:
            print('epoch={} | loss={} | metric={}'.format(epoch,loss,metric))
train_model(model, 500)

epoch=100 | loss=0.24013393647928752 | metric=0.9199999916553497
epoch=200 | loss=0.2614943907559609 | metric=0.9234999921917916
epoch=300 | loss=0.33852576003051127 | metric=0.9229999913275242
epoch=400 | loss=0.3222889948804772 | metric=0.9242499923706055
epoch=500 | loss=0.29889976761380693 | metric=0.924999992698431


In [114]:
# 预测
# X_pred = torch.squeeze(model.forward(X))
Xp_pred = X[torch.squeeze(model.forward(X)>=0.5)]
Xn_pred = X[torch.squeeze(model.forward(X)<0.5)]
Xp_pred

tensor([[-2.1195,  4.4397],
        [ 3.2330, -5.0898],
        [-4.6811,  2.3082],
        ...,
        [-5.1976, -3.7398],
        [-3.8134, -5.3091],
        [-0.3192, -6.3915]])

In [115]:
# 预览
from plotly.subplots import make_subplots

fig = go.Figure()

trace_Xp = go.Scatter(x=Xp[:, 0], y=Xp[:, 1], mode='markers', name='p')
trace_Xn = go.Scatter(x=Xn[:, 0], y=Xn[:, 1], mode='markers', name='n')
trace_Xp_pred = go.Scatter(x=Xp_pred[:, 0], y=Xp_pred[:, 1], mode='markers', name='p_pre')
trace_Xn_pred = go.Scatter(x=Xn_pred[:, 0], y=Xn_pred[:, 1], mode='markers', name='n_pre')


fig = make_subplots(rows=1,  # 将画布分为两行
            cols=2,  # 将画布分为两列
            subplot_titles=['y_true','y_pred'],# 子图的标题
            x_title="x",
            y_title="y"
            )

fig.add_trace(trace_Xp,1,1)
fig.add_trace(trace_Xn,1,1)
fig.add_trace(trace_Xp_pred,1,2)
fig.add_trace(trace_Xn_pred,1,2)

fig.show()

# 高阶API

In [116]:
print(X.shape)
print(Y.shape)

torch.Size([4000, 2])
torch.Size([4000, 1])


In [117]:
ds = TensorDataset(X,Y)
split_ration = 0.7
ds_train,ds_valid = torch.utils.data.random_split(ds,[int(len(ds)*split_ration),len(ds)-int(len(ds)*split_ration)])
dl_train = DataLoader(ds_train,batch_size = 100,shuffle=True,num_workers=2)
dl_valid = DataLoader(ds_valid,batch_size = 100,num_workers=2)

In [125]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(2,4)
        self.fc2 = nn.Linear(4,8)
        self.fc3 = nn.Linear(8,1)
    
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        y = nn.Sigmoid()(self.fc3(x))
        return y


In [126]:
# !pip install torchkeras
import torchkeras

model = torchkeras.Model(Net())
model.summary(input_shape =(2,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 4]              12
            Linear-2                    [-1, 8]              40
            Linear-3                    [-1, 1]               9
Total params: 61
Trainable params: 61
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.000008
Forward/backward pass size (MB): 0.000099
Params size (MB): 0.000233
Estimated Total Size (MB): 0.000340
----------------------------------------------------------------


In [132]:
def accuracy(y_pred, y_true):
    y_pred = torch.where(y_pred>0.5, torch.ones_like(y_pred,dtype=torch.float32),torch.zeros_like(y_pred,dtype=torch.float32,))
    acc = torch.mean(1-torch.abs(y_pred-y_true))
    return acc

model.compile(loss_func=nn.BCELoss(),optimizer=torch.optim.Adam(model.parameters(), lr=0.01), metrics_dict={'accuracy':accuracy})
dfhistory = model.fit(100,dl_train=dl_train,dl_val=dl_valid,log_step_freq=20)

Start Training ...

================================================================================2021-12-01 03:09:44
{'step': 20, 'loss': 0.166, 'accuracy': 0.934}

 +-------+-------+----------+----------+--------------+
| epoch |  loss | accuracy | val_loss | val_accuracy |
+-------+-------+----------+----------+--------------+
|   1   | 0.173 |  0.932   |  0.205   |    0.923     |
+-------+-------+----------+----------+--------------+

================================================================================2021-12-01 03:09:44
{'step': 20, 'loss': 0.174, 'accuracy': 0.932}

 +-------+------+----------+----------+--------------+
| epoch | loss | accuracy | val_loss | val_accuracy |
+-------+------+----------+----------+--------------+
|   2   | 0.17 |  0.933   |  0.204   |    0.924     |
+-------+------+----------+----------+--------------+

================================================================================2021-12-01 03:09:44
{'step': 20, 'loss': 0.167, 'accura

In [133]:
# 预测
# X_pred = torch.squeeze(model.forward(X))
Xp_pred = X[torch.squeeze(model.forward(X)>=0.5)]
Xn_pred = X[torch.squeeze(model.forward(X)<0.5)]
Xp_pred

tensor([[-2.1195,  4.4397],
        [ 3.2330, -5.0898],
        [-4.6811,  2.3082],
        ...,
        [ 2.8623,  6.0141],
        [ 5.6765,  2.6894],
        [-2.6837,  5.4794]])

In [134]:
# 预览
from plotly.subplots import make_subplots

fig = go.Figure()

trace_Xp = go.Scatter(x=Xp[:, 0], y=Xp[:, 1], mode='markers', name='p')
trace_Xn = go.Scatter(x=Xn[:, 0], y=Xn[:, 1], mode='markers', name='n')
trace_Xp_pred = go.Scatter(x=Xp_pred[:, 0], y=Xp_pred[:, 1], mode='markers', name='p_pre')
trace_Xn_pred = go.Scatter(x=Xn_pred[:, 0], y=Xn_pred[:, 1], mode='markers', name='n_pre')


fig = make_subplots(rows=1,  # 将画布分为两行
            cols=2,  # 将画布分为两列
            subplot_titles=['y_true','y_pred'],# 子图的标题
            x_title="x",
            y_title="y"
            )

fig.add_trace(trace_Xp,1,1)
fig.add_trace(trace_Xn,1,1)
fig.add_trace(trace_Xp_pred,1,2)
fig.add_trace(trace_Xn_pred,1,2)

fig.show()

In [135]:
# 模型评估
dfhistory

,loss,accuracy,val_loss,val_accuracy
0,0.172960,0.932143,0.205449,0.922500
1,0.169799,0.932857,0.203516,0.924167
2,0.169343,0.933214,0.212958,0.915833
3,0.173387,0.929286,0.206057,0.921667
4,0.170659,0.932500,0.214123,0.917500
...,...,...,...,...
95,0.168780,0.932857,0.211268,0.920000
96,0.173228,0.931786,0.203098,0.924167
97,0.167551,0.937143,0.203943,0.924167
98,0.167584,0.931429,0.205625,0.925000


In [136]:
# 使用模型
model.predict(dl_valid)

tensor([[0.0010],
        [0.9961],
        [0.0965],
        ...,
        [0.9778],
        [0.9886],
        [0.9926]])